In [1]:
# MarginBased_model.py
from mxnet import gluon
from mxnet.gluon import Block, HybridBlock
import numpy as np

class L2Normalization(HybridBlock):
    r"""Applies L2 Normalization to input.
    Parameters
    ----------
    mode : str
        Mode of normalization.
        See :func:`~mxnet.ndarray.L2Normalization` for available choices.
    Inputs:
        - **data**: input tensor with arbitrary shape.
    Outputs:
        - **out**: output tensor with the same shape as `data`.
    """
    def __init__(self, mode, **kwargs):
        self._mode = mode
        super(L2Normalization, self).__init__(**kwargs)

    def hybrid_forward(self, F, x):
        return F.L2Normalization(x, mode=self._mode, name='l2_norm')

    def __repr__(self):
        s = '{name}({_mode})'
        return s.format(name=self.__class__.__name__,
                        **self.__dict__)


def get_distance(F, x):
    """Helper function for margin-based loss. Return a distance matrix given a matrix."""
    n = x.shape[0]

    square = F.sum(x ** 2.0, axis=1, keepdims=True)
    distance_square = square + square.transpose() - (2.0 * F.dot(x, x.transpose()))

    # Adding identity to make sqrt work.
    return F.sqrt(distance_square + F.array(np.identity(n)))

class DistanceWeightedSampling(HybridBlock):
    r"""Distance weighted sampling. See "sampling matters in deep embedding learning"
    paper for details.
    Parameters
    ----------
    batch_k : int
        Number of images per class.
    Inputs:
        - **data**: input tensor with shape (batch_size, embed_dim).
        Here we assume the consecutive batch_k examples are of the same class.
        For example, if batch_k = 5, the first 5 examples belong to the same class,
        6th-10th examples belong to another class, etc.
    Outputs:
        - a_indices: indices of anchors.
        - x[a_indices]: sampled anchor embeddings.
        - x[p_indices]: sampled positive embeddings.
        - x[n_indices]: sampled negative embeddings.
        - x: embeddings of the input batch.
    """
    def __init__(self, batch_k, cutoff=0.5, nonzero_loss_cutoff=1.4, **kwargs):
        self.batch_k = batch_k
        self.cutoff = cutoff

        # We sample only from negatives that induce a non-zero loss.
        # These are negatives with a distance < nonzero_loss_cutoff.
        # With a margin-based loss, nonzero_loss_cutoff == margin + beta.
        self.nonzero_loss_cutoff = nonzero_loss_cutoff
        super(DistanceWeightedSampling, self).__init__(**kwargs)

    def hybrid_forward(self, F, x):
        k = self.batch_k
        n, d = x.shape

        distance = get_distance(F, x)
        # Cut off to avoid high variance.
        distance = F.maximum(distance, self.cutoff)

        # Subtract max(log(distance)) for stability.
        log_weights = ((2.0 - float(d)) * F.log(distance)
                       - (float(d - 3) / 2) * F.log(1.0 - 0.25 * (distance ** 2.0)))
        weights = F.exp(log_weights - F.max(log_weights))

        # Sample only negative examples by setting weights of
        # the same-class examples to 0.
        mask = np.ones(weights.shape)
        for i in range(0, n, k):
            mask[i:i+k, i:i+k] = 0

        weights = weights * F.array(mask) * (distance < self.nonzero_loss_cutoff)
        weights = weights / F.sum(weights, axis=1, keepdims=True)

        a_indices = []
        p_indices = []
        n_indices = []

        np_weights = weights.asnumpy()
        for i in range(n):
            block_idx = i // k

            try:
                n_indices += np.random.choice(n, k-1, p=np_weights[i]).tolist()
            except:
                n_indices += np.random.choice(n, k-1).tolist()
            for j in range(block_idx * k, (block_idx + 1) * k):
                if j != i:
                    a_indices.append(i)
                    p_indices.append(j)

        return a_indices, x[a_indices], x[p_indices], x[n_indices], x

    def __repr__(self):
        s = '{name}({batch_k})'
        return s.format(name=self.__class__.__name__,
                        **self.__dict__)


class MarginNet(Block):
    r"""Embedding network with distance weighted sampling.
    It takes a base CNN and adds an embedding layer and a
    sampling layer at the end.
    Parameters
    ----------
    base_net : Block
        Base network.
    emb_dim : int
        Dimensionality of the embedding.
    batch_k : int
        Number of images per class in a batch. Used in sampling.
    Inputs:
        - **data**: input tensor with shape (batch_size, channels, width, height).
        Here we assume the consecutive batch_k images are of the same class.
        For example, if batch_k = 5, the first 5 images belong to the same class,
        6th-10th images belong to another class, etc.
    Outputs:
        - The output of DistanceWeightedSampling.
    """
    def __init__(self, base_net, emb_dim, batch_k, **kwargs):
        super(MarginNet, self).__init__(**kwargs)
        with self.name_scope():
            self.base_net = base_net
            self.dense = gluon.nn.Dense(emb_dim)
            self.normalize = L2Normalization(mode='instance')
            self.sampled = DistanceWeightedSampling(batch_k=batch_k)

    def forward(self, x):
        z = self.base_net(x)
        z = self.dense(z)
        z = self.normalize(z)
        z = self.sampled(z)
        return z


class MarginLoss(gluon.loss.Loss):
    r"""Margin based loss.
    Parameters
    ----------
    margin : float
        Margin between positive and negative pairs.
    nu : float
        Regularization parameter for beta.
    Inputs:
        - anchors: sampled anchor embeddings.
        - positives: sampled positive embeddings.
        - negatives: sampled negative embeddings.
        - beta_in: class-specific betas.
        - a_indices: indices of anchors. Used to get class-specific beta.
    Outputs:
        - Loss.
    """
    def __init__(self, margin=0.2, nu=0.0, weight=None, batch_axis=0, **kwargs):
        super(MarginLoss, self).__init__(weight, batch_axis, **kwargs)
        self._margin = margin
        self._nu = nu

    def hybrid_forward(self, F, anchors, positives, negatives, beta_in, a_indices=None):
        if a_indices is not None:
            # Jointly train class-specific beta.
            beta = beta_in.data()[a_indices]
            beta_reg_loss = F.sum(beta) * self._nu
        else:
            # Use a constant beta.
            beta = beta_in
            beta_reg_loss = 0.0

        d_ap = F.sqrt(F.sum(F.square(positives - anchors), axis=1) + 1e-8)
        d_an = F.sqrt(F.sum(F.square(negatives - anchors), axis=1) + 1e-8)

        pos_loss = F.maximum(d_ap - beta + self._margin, 0.0)
        neg_loss = F.maximum(beta - d_an + self._margin, 0.0)

        pair_cnt = float(F.sum((pos_loss > 0.0) + (neg_loss > 0.0)).asscalar())

        # Normalize based on the number of pairs.
        loss = (F.sum(pos_loss + neg_loss) + beta_reg_loss) / pair_cnt
        print('loss',loss)
        return gluon.loss._apply_weighting(F, loss, self._weight, None)

In [9]:
#!pip install mxnet

    100% |################################| 23.8MB 869kB/s eta 0:00:01  7% |##                              | 1.9MB 17.6MB/s eta 0:00:02    22% |#######                         | 5.3MB 27.5MB/s eta 0:00:01    44% |##############                  | 10.6MB 14.0MB/s eta 0:00:01    51% |################                | 12.2MB 16.1MB/s eta 0:00:01    54% |#################               | 13.1MB 16.7MB/s eta 0:00:01    62% |####################            | 14.9MB 14.2MB/s eta 0:00:01    70% |######################          | 16.9MB 22.8MB/s eta 0:00:01    94% |##############################  | 22.4MB 25.6MB/s eta 0:00:01
grpcio 1.11.0 has requirement protobuf>=3.5.0.post1, but you'll have protobuf 3.4.0 which is incompatible.


In [2]:
from __future__ import division

import argparse
import logging
import time
import os
import numpy as np
from tqdm import tqdm
from bottleneck import argpartition
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
from torch.autograd import Function
import torch.backends.cudnn as cudnn
from LFWDataset import LFWDataset
import mxnet as mx
#from data import cub200_iterator
from mxnet import gluon
from mxnet.gluon.model_zoo import vision as models
from mxnet import autograd as ag, nd
#from model import MarginNet, MarginLoss

In [3]:


logging.basicConfig(level=logging.INFO)

# CLI
parser = argparse.ArgumentParser(description='train a model for image classification.')
parser.add_argument('--data-path', type=str, default='/scratch/ys3225/deeplearningdataset', #/scratch/hb1500/deeplearningdataset/train
                    help='path of data.')

###########
parser.add_argument('--data_train-path', type=str, default='/scratch/ys3225/deeplearningdataset/train', 
                    help='path of data.')
parser.add_argument('--data_test-path', type=str, default='/scratch/ys3225/deeplearningdataset/test', 
                    help='path of data.')
###########

parser.add_argument('--embed-dim', type=int, default=128,
                    help='dimensionality of image embedding. default is 128.')
parser.add_argument('--batch-size', type=int, default=70,
                    help='training batch size per device (CPU/GPU). default is 70.')
parser.add_argument('--batch-k', type=int, default=5,
                    help='number of images per class in a batch. default is 5.')
parser.add_argument('--gpus', type=str, default='',
                    help='list of gpus to use, e.g. 0 or 0,2,5. empty means using cpu.')
parser.add_argument('--epochs', type=int, default=20,
                    help='number of training epochs. default is 20.')
parser.add_argument('--optimizer', type=str, default='adam',
                    help='optimizer. default is adam.')
parser.add_argument('--lr', type=float, default=0.0001,
                    help='learning rate. default is 0.0001.')
parser.add_argument('--lr-beta', type=float, default=0.1,
                    help='learning rate for the beta in margin based loss. default is 0.1.')
parser.add_argument('--margin', type=float, default=0.2,
                    help='margin for the margin based loss. default is 0.2.')
parser.add_argument('--beta', type=float, default=1.2,
                    help='initial value for beta. default is 1.2.')
parser.add_argument('--nu', type=float, default=0.0,
                    help='regularization parameter for beta. default is 0.0.')
parser.add_argument('--factor', type=float, default=0.5,
                    help='learning rate schedule factor. default is 0.5.')
parser.add_argument('--steps', type=str, default='12,14,16,18',
                    help='epochs to update learning rate. default is 12,14,16,18.')
parser.add_argument('--wd', type=float, default=0.0001,
                    help='weight decay rate. default is 0.0001.')
parser.add_argument('--seed', type=int, default=123,
                    help='random seed to use. default=123.')
parser.add_argument('--model', type=str, default='resnet18_v1',
                    help='type of model to use. see vision_model for options.resnet50_v2')
parser.add_argument('--save-model-prefix', type=str, default='margin_loss_model',
                    help='prefix of models to be saved.')
parser.add_argument('--use-pretrained', action='store_true',
                    help='enable using pretrained model from gluon.')
parser.add_argument('--kvstore', type=str, default='device',
                    help='kvstore to use for trainer.')
parser.add_argument('--log-interval', type=int, default=20,
                    help='number of batches to wait before logging.')

###################################
parser.add_argument('--dataroot', type=str, default='/scratch/ys3225/deeplearningdataset/train',#default='/scratch/hb1500/deeplearningdataset/train'
                    help='path to dataset')
parser.add_argument('--testdataroot', type=str, default='/scratch/ys3225/deeplearningdataset/test',#default='/media/lior/LinuxHDD/datasets/vgg_face_dataset/aligned'
                    help='path to dataset')
parser.add_argument('--lfw-dir', type=str, default='/scratch/ys3225/lfw',
                    help='path to dataset')
parser.add_argument('--lfw-pairs-path', type=str, default='lfw_pairs.txt',
                    help='path to pairs file')

parser.add_argument('--log-dir', default='/scratch/ys3225/logdir_triplet_loss',
                    help='folder to output model checkpoints')
parser.add_argument('--resume',
                    default='/scratch/ys3225/resume/run-optim_adam-lr0.001-wd0.0-embeddings512-center0.5-MSCeleb/checkpoint_11.pth',
                    type=str, metavar='PATH',
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
# parser.add_argument('--epochs', type=int, default=50, metavar='E',
#                     help='number of epochs to train (default: 10)')
# Training options
# parser.add_argument('--embedding-size', type=int, default=256, metavar='ES',
#                     help='Dimensionality of the embedding')

parser.add_argument('--center_loss_weight', type=float, default=0.5, help='weight for center loss')
parser.add_argument('--alpha', type=float, default=0.5, help='learning rate of the centers')
parser.add_argument('--embedding-size', type=int, default=512, metavar='ES',
                    help='Dimensionality of the embedding')

# parser.add_argument('--batch-size', type=int, default=64, metavar='BS',
#                     help='input batch size for training (default: 128)')
parser.add_argument('--test-batch-size', type=int, default=64, metavar='BST',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--n-triplets', type=int, default=1000000, metavar='N',
                    help='how many triplets will generate from the dataset,default=1000000')
# parser.add_argument('--margin', type=float, default=1.0, metavar='MARGIN',
#                     help='the margin value for the triplet loss function (default: 1.0')
# parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
#                     help='learning rate (default: 0.001)')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')

parser.add_argument('--lr-decay', default=1e-4, type=float, metavar='LRD',
                    help='learning rate decay ratio (default: 1e-4')
# parser.add_argument('--wd', default=0.0, type=float,
#                     metavar='W', help='weight decay (default: 0.0)')
# parser.add_argument('--optimizer', default='adam', type=str,
#                     metavar='OPT', help='The optimizer to use (default: Adagrad)')
# Device options
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--gpu-id', default='0', type=str,
                    help='id(s) for CUDA_VISIBLE_DEVICES')
# parser.add_argument('--seed', type=int, default=0, metavar='S',
#                     help='random seed (default: 0)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='LI',
#                     help='how many batches to wait before logging training status')
###################################
opt = parser.parse_args(args = [])

logging.info(opt)

INFO:root:Namespace(alpha=0.5, batch_k=5, batch_size=70, beta=1.2, beta1=0.5, center_loss_weight=0.5, data_path='/scratch/ys3225/deeplearningdataset', data_test_path='/scratch/ys3225/deeplearningdataset/test', data_train_path='/scratch/ys3225/deeplearningdataset/train', dataroot='/scratch/ys3225/deeplearningdataset/train', embed_dim=128, embedding_size=512, epochs=20, factor=0.5, gpu_id='0', gpus='', kvstore='device', lfw_dir='/scratch/ys3225/lfw', lfw_pairs_path='lfw_pairs.txt', log_dir='/scratch/ys3225/logdir_triplet_loss', log_interval=20, lr=0.0001, lr_beta=0.1, lr_decay=0.0001, margin=0.2, model='resnet18_v1', n_triplets=1000000, no_cuda=False, nu=0.0, optimizer='adam', resume='/scratch/ys3225/resume/run-optim_adam-lr0.001-wd0.0-embeddings512-center0.5-MSCeleb/checkpoint_11.pth', save_model_prefix='margin_loss_model', seed=123, start_epoch=0, steps='12,14,16,18', test_batch_size=64, testdataroot='/scratch/ys3225/deeplearningdataset/test', use_pretrained=False, wd=0.0001)


In [4]:
# Settings.
mx.random.seed(opt.seed)
np.random.seed(opt.seed)

batch_size = opt.batch_size

gpus = [] if opt.gpus is None or opt.gpus is '' else [
    int(gpu) for gpu in opt.gpus.split(',')]
num_gpus = len(gpus)

batch_size *= max(1, num_gpus)
context = [mx.gpu(i) for i in gpus] if num_gpus > 0 else [mx.cpu()]
steps = [int(step) for step in opt.steps.split(',')]

# Construct model.
kwargs = {'ctx': context, 'pretrained': opt.use_pretrained}
net = models.get_model(opt.model, **kwargs)

if opt.use_pretrained:
    # Use a smaller learning rate for pre-trained convolutional layers.
    for v in net.collect_params().values():
        if 'conv' in v.name:
            setattr(v, 'lr_mult', 0.01)

net.hybridize()
net = MarginNet(net.features, opt.embed_dim, opt.batch_k)
beta = mx.gluon.Parameter('beta', shape=(100,))

In [5]:
args = opt
# set the device to use by setting CUDA_VISIBLE_DEVICES env variable in
# order to prevent any memory allocation on unused GPUs
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_id

args.cuda = not args.no_cuda and torch.cuda.is_available()
np.random.seed(args.seed)

if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)

if args.cuda:
    cudnn.benchmark = True

#LOG_DIR = args.log_dir + '/run-optim_{}-lr{}-wd{}-embeddings{}-center_loss{}-MSCeleb'.format(args.optimizer, args.lr, args.wd,args.embedding_size,args.center_loss_weight)
LOG_DIR = args.log_dir + '/run-optim_{}-n{}-lr{}-wd{}-m{}-embeddings{}-msceleb-alpha10'\
    .format(args.optimizer, args.n_triplets, args.lr, args.wd,
            args.margin,args.embedding_size)

# create logger
#logger = Logger(LOG_DIR)

kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}
#l2_dist = PairwiseDistance(2)


transform = transforms.Compose([
                         transforms.Resize(96),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),
                         transforms.Normalize(mean = [ 0.5, 0.5, 0.5 ],
                                               std = [ 0.5, 0.5, 0.5 ])
                     ])

#train_dir = TripletFaceDataset(args.dataroot,transform=transform)
#train_dir = TripletFaceDataset(dir=args.dataroot,n_triplets=args.n_triplets,transform=transform)
#train_dir = MarginDataset(dir=args.dataroot,n_triplets=args.n_triplets,transform=transform)
train_dir = ImageFolder(args.dataroot,transform=transform)
train_loader = torch.utils.data.DataLoader(train_dir,batch_size=args.batch_size, shuffle=False, **kwargs)
testacc_dir = ImageFolder(args.testdataroot,transform=transform)
test_loader = torch.utils.data.DataLoader(
    LFWDataset(dir=args.lfw_dir,pairs_path=args.lfw_pairs_path,
                     transform=transform),
    batch_size=args.batch_size, shuffle=False, **kwargs)
testaccuracy_loader = torch.utils.data.DataLoader(testacc_dir,
    batch_size=args.batch_size, shuffle=True, **kwargs)



100%|██████████| 6000/6000 [00:05<00:00, 1099.02it/s]


In [6]:

# Get iterators.
#train_data, val_data = cub200_iterator(opt.data_path, opt.batch_k, batch_size, (3, 224, 224))


def get_distance_matrix(x):
    """Get distance matrix given a matrix. Used in testing."""
    square = nd.sum(x ** 2.0, axis=1, keepdims=True)
    distance_square = square + square.transpose() - (2.0 * nd.dot(x, x.transpose()))
    return nd.sqrt(distance_square)


def evaluate_emb(emb, labels):
    """Evaluate embeddings based on Recall@k."""
    d_mat = get_distance_matrix(emb)
    d_mat = d_mat.asnumpy()
    labels = labels.asnumpy()

    names = []
    accs = []
    for k in [1, 2, 4, 8, 16]:
        names.append('Recall@%d' % k)
        correct, cnt = 0.0, 0.0
        for i in range(emb.shape[0]):
            d_mat[i, i] = 1e10
            nns = argpartition(d_mat[i], k)[:k]
            if any(labels[i] == labels[nn] for nn in nns):
                correct += 1
            cnt += 1
        accs.append(correct/cnt)
    return names, accs


def test(ctx):
    """Test a model."""
    val_data.reset()
    outputs = []
    labels = []
    for batch in val_data:
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        for x in data:
            outputs.append(net(x)[-1])
        labels += label

    outputs = nd.concatenate(outputs, axis=0)[:val_data.n_test]
    labels = nd.concatenate(labels, axis=0)[:val_data.n_test]
    return evaluate_emb(outputs, labels)


def get_lr(lr, epoch, steps, factor):
    """Get learning rate based on schedule."""
    for s in steps:
        if epoch >= s:
            lr *= factor
    return lr


def train(epochs, ctx):
    """Training function."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    net.initialize(mx.init.Xavier(magnitude=2), ctx=ctx)

    opt_options = {'learning_rate': opt.lr, 'wd': opt.wd}
    if opt.optimizer == 'sgd':
        opt_options['momentum'] = 0.9
    if opt.optimizer == 'adam':
        opt_options['epsilon'] = 1e-7
    trainer = gluon.Trainer(net.collect_params(), opt.optimizer,
                            opt_options,
                            kvstore=opt.kvstore)
    if opt.lr_beta > 0.0:
        # Jointly train class-specific beta.
        # See "sampling matters in deep embedding learning" paper for details.
        beta.initialize(mx.init.Constant(opt.beta), ctx=ctx)
        trainer_beta = gluon.Trainer([beta], 'sgd',
                                     {'learning_rate': opt.lr_beta, 'momentum': 0.9},
                                     kvstore=opt.kvstore)

    loss = MarginLoss(margin=opt.margin, nu=opt.nu)

    best_val = 0.0
    for epoch in range(epochs):
        tic = time.time()
        prev_loss, cumulative_loss = 0.0, 0.0

        # Learning rate schedule.
        trainer.set_learning_rate(get_lr(opt.lr, epoch, steps, opt.factor))
        logging.info('Epoch %d learning rate=%f', epoch, trainer.learning_rate)
        if opt.lr_beta > 0.0:
            trainer_beta.set_learning_rate(get_lr(opt.lr_beta, epoch, steps, opt.factor))
            logging.info('Epoch %d beta learning rate=%f', epoch, trainer_beta.learning_rate)
            
            
        pbar = tqdm(enumerate(train_loader))
        labels, distances = [], []
            
            
        # Inner training loop.
        #for i in range(200):
        for batch_idx, (data,label) in pbar:
#             batch = train_data.next()
#             data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
#             label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)



#         data_v = Variable(data.cuda())
#         target_var = Variable(label)
            #Ls = []
            with ag.record():
                x = mx.ndarray.array(data.numpy())
                y = mx.ndarray.array(label.numpy())
                a_indices, anchors, positives, negatives, _ = net(x)

                if opt.lr_beta > 0.0:
                    L = loss(anchors, positives, negatives, beta, y[a_indices])
                else:
                    L = loss(anchors, positives, negatives, opt.beta, None)

                # Store the loss and do backward after we have done forward
                # on all GPUs for better speed on multiple GPUs.
                #Ls.append(L)
                cumulative_loss += nd.mean(L).asscalar()

            #for L in Ls:
                L.backward()

            # Update.
            trainer.step(x.shape[0])
            if opt.lr_beta > 0.0:
                trainer_beta.step(x.shape[0])

            if (batch_idx) % opt.log_interval == 0:
                #print(cumulative_loss)
                logging.info('[Epoch %d, Iter %d] training loss=%f' % (
                    epoch, batch_idx, cumulative_loss - prev_loss))
                prev_loss = cumulative_loss

        logging.info('[Epoch %d] training loss=%f'%(epoch, cumulative_loss))
        logging.info('[Epoch %d] time cost: %f'%(epoch, time.time()-tic))

        names, val_accs = test(ctx)
        for name, val_acc in zip(names, val_accs):
            logging.info('[Epoch %d] validation: %s=%f'%(epoch, name, val_acc))

        if val_accs[0] > best_val:
            best_val = val_accs[0]
            logging.info('Saving %s.' % opt.save_model_prefix)
            net.collect_params().save('%s.params' % opt.save_model_prefix)
    return best_val

In [17]:
if __name__ == '__main__':
    best_val_recall = train(opt.epochs, context)
    print('Best validation Recall@1: %.2f.' % best_val_recall)

/home/ys3225/py3.6.3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter resnetv10_conv0_weight is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/ys3225/py3.6.3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter resnetv10_batchnorm0_gamma is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/ys3225/py3.6.3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter resnetv10_batchnorm0_beta is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/ys3225/py3.6.3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter resnetv10_batchnorm0_running_mean is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=T

INFO:root:Epoch 0 beta learning rate=0.100000




0it [00:00, ?it/s]INFO:root:[Epoch 0, Iter 0] training loss=0.885826




1it [00:03,  3.77s/it]



2it [00:11,  5.73s/it]



3it [00:19,  6.44s/it]



4it [00:27,  6.77s/it]



5it [00:34,  6.95s/it]



6it [00:42,  7.08s/it]



7it [00:51,  7.31s/it]



8it [00:58,  7.36s/it]



9it [01:06,  7.40s/it]



10it [01:14,  7.43s/it]



11it [01:22,  7.46s/it]



12it [01:29,  7.48s/it]



13it [01:37,  7.49s/it]



14it [01:45,  7.51s/it]



15it [01:52,  7.52s/it]



16it [02:00,  7.53s/it]



17it [02:08,  7.54s/it]



18it [02:15,  7.55s/it]



19it [02:23,  7.56s/it]Process Process-9:
Process Process-10:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/share/apps/pyth

KeyboardInterrupt: 

In [ ]:
ctx =context
epochs = 1
"""Training function."""
if isinstance(ctx, mx.Context):
    ctx = [ctx]
net.initialize(mx.init.Xavier(magnitude=2), ctx=ctx)

opt_options = {'learning_rate': opt.lr, 'wd': opt.wd}
if opt.optimizer == 'sgd':
    opt_options['momentum'] = 0.9
if opt.optimizer == 'adam':
    opt_options['epsilon'] = 1e-7
trainer = gluon.Trainer(net.collect_params(), opt.optimizer,
                        opt_options,
                        kvstore=opt.kvstore)
if opt.lr_beta > 0.0:
    # Jointly train class-specific beta.
    # See "sampling matters in deep embedding learning" paper for details.
    beta.initialize(mx.init.Constant(opt.beta), ctx=ctx)
    trainer_beta = gluon.Trainer([beta], 'sgd',
                                 {'learning_rate': opt.lr_beta, 'momentum': 0.9},
                                 kvstore=opt.kvstore)

loss = MarginLoss(margin=opt.margin, nu=opt.nu)

best_val = 0.0
for epoch in range(epochs):
    tic = time.time()
    prev_loss, cumulative_loss = 0.0, 0.0

    # Learning rate schedule.
    trainer.set_learning_rate(get_lr(opt.lr, epoch, steps, opt.factor))
    logging.info('Epoch %d learning rate=%f', epoch, trainer.learning_rate)
    if opt.lr_beta > 0.0:
        trainer_beta.set_learning_rate(get_lr(opt.lr_beta, epoch, steps, opt.factor))
        logging.info('Epoch %d beta learning rate=%f', epoch, trainer_beta.learning_rate)


    pbar = tqdm(enumerate(train_loader))
    labels, distances = [], []


    # Inner training loop.
    #for i in range(200):
    for batch_idx, (data,label) in pbar:
#             batch = train_data.next()
#             data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
#             label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)



#         data_v = Variable(data.cuda())
#         target_var = Variable(label)
        #Ls = []
        with ag.record():
            x = mx.ndarray.array(data.numpy())
            y = mx.ndarray.array(label.numpy())
            a_indices, anchors, positives, negatives, _ = net(x)

            if opt.lr_beta > 0.0:
                L = loss(anchors, positives, negatives, beta, y[a_indices])
            else:
                L = loss(anchors, positives, negatives, opt.beta, None)
            #print('L',L)
            # Store the loss and do backward after we have done forward
            # on all GPUs for better speed on multiple GPUs.
            #Ls.append(L)
            cumulative_loss += nd.mean(L).asscalar()

        #for L in Ls:
            L.backward()

        # Update.
        trainer.step(x.shape[0])
        if opt.lr_beta > 0.0:
            trainer_beta.step(x.shape[0])

        if (batch_idx) % opt.log_interval == 0:
            #print('cumulative_loss',cumulative_loss)
            logging.info('[Epoch %d, Iter %d] training loss=%f' % (
                epoch, batch_idx, cumulative_loss - prev_loss))
            prev_loss = cumulative_loss

    logging.info('[Epoch %d] training loss=%f'%(epoch, cumulative_loss))
    logging.info('[Epoch %d] time cost: %f'%(epoch, time.time()-tic))

    names, val_accs = test(ctx)
    for name, val_acc in zip(names, val_accs):
        logging.info('[Epoch %d] validation: %s=%f'%(epoch, name, val_acc))

    if val_accs[0] > best_val:
        best_val = val_accs[0]
        logging.info('Saving %s.' % opt.save_model_prefix)
        net.collect_params().save('%s.params' % opt.save_model_prefix)


INFO:root:Epoch 0 learning rate=0.000100
INFO:root:Epoch 0 beta learning rate=0.100000
0it [00:00, ?it/s]INFO:root:[Epoch 0, Iter 0] training loss=0.930200
1it [00:11, 11.17s/it]

loss 
[ 0.93019962]
<NDArray 1 @cpu(0)>
L 
[ 0.93019962]
<NDArray 1 @cpu(0)>
cumulative_loss 0.930199623108


2it [00:24, 12.08s/it]

loss 
[ 0.91740507]
<NDArray 1 @cpu(0)>
L 
[ 0.91740507]
<NDArray 1 @cpu(0)>


3it [00:31, 10.59s/it]

...

loss 
[ 0.24296364]
<NDArray 1 @cpu(0)>
L 
[ 0.24296364]
<NDArray 1 @cpu(0)>


139it [18:00,  7.77s/it]

loss 
[ 0.19890828]
<NDArray 1 @cpu(0)>
L 
[ 0.19890828]
<NDArray 1 @cpu(0)>


140it [18:07,  7.77s/it]

loss 
[ 0.20485149]
<NDArray 1 @cpu(0)>
L 
[ 0.20485149]
<NDArray 1 @cpu(0)>
